# Calibration

This section of the tutorial will cover the calibration module of Mitiq. 

Before running this tutorial, make sure you have `mitiq` installed.
```
pip install mitiq
```
Use `pip list | grep mitiq` to ensure it is installed.

In [18]:
# If you have not already done so, you can instll mitiq by uncommenting out the line below
# %pip install mitiq

In [48]:
from mitiq import (
    Calibrator,
    MeasurementResult
)

import numpy as np
import cirq
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2  # Fake (simulated) QPU

## Executor for Calibration
When using the calibration module to test different mitigation techniques, you must define an executor that returns measured bitstrings. 

In [36]:
def execute_cirq_with_noise(circuit, noise_level=0.001):
    """Return bitstrings outcomes from circuit executions performed by
    a density matrix simulation with depolarizing noise."""
    circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [21]:
def execute_qiskit_FakeJakartaV2(circuit, shots=100):
    """Return bitstrings outcomes from circuit executions performed by
    the FakeJakartaV2 simulator with noise."""
    noisy_backend = FakeJakartaV2()
    noisy_result = noisy_backend.run(circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(circuit)
    noisy_counts = { k.replace(" ",""):v for k, v in noisy_counts.items()}
    measurements = MeasurementResult.from_counts(noisy_counts)
    return measurements

## Run the Calibrator
Now that the executor is defined, we can instantiate the Calibrator by passing in the executor along with a `fronted`.

In [37]:
from mitiq import SUPPORTED_PROGRAM_TYPES
SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [42]:
cal = Calibrator(execute_cirq_with_noise, frontend="cirq")
# cal = Calibrator(execute_qiskit_FakeJakartaV2, frontend="qiskit")
cal.run(log="flat")

┌──────────────────────────┬────────────────────────────────────┬────────────────────────────┐
│ benchmark                │ strategy                           │ performance                │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.03          │
│ Circuit depth: 2         │ Scale factors: 1.0, 3.0, 5.0       │ Mitigated error: 0.01      │
│ Two qubit gate count: 1  │ Scale method: fold_gates_at_random │ Improvement factor: 3.0    │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.03          │
│ Circuit depth: 2         │ Scale factors: 1.0, 2

/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/mitiq/calibration/calibrator.py:76: RuntimeWarning: divide by zero encountered in scalar divide
  f"Improvement factor: {round(noisy_error / mitigated_error, 4)}"


In [40]:
cal.run(log="cartesian")

┌────────────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────────────────┬────────────────────────────┐
│ strategy\benchmark                 │ Type: ghz                  │ Type: w                    │ Type: rb                               │ Type: mirror               │
│                                    │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2                          │ Num qubits: 2              │
│                                    │ Circuit depth: 2           │ Circuit depth: 2           │ Circuit depth: 48                      │ Circuit depth: 33          │
│                                    │ Two qubit gate count: 1    │ Two qubit gate count: 2    │ Two qubit gate count: 11               │ Two qubit gate count: 14   │
├────────────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────────────────┼────────────────────────────

In [49]:
cal.best_strategy()

{'technique': 'PEC', 'representation_function': 'represent_operation_with_local_depolarizing_noise', 'noise_level': 0.001, 'noise_bias': 0, 'is_qubit_dependent': False, 'num_samples': 200}

# SETTINGS
The settings parameter in the Calibrator allows for specifying details about circuit and technique.

In [50]:
from mitiq.calibration import PEC_SETTINGS

cal = Calibrator(execute_cirq_with_noise, frontend="cirq", settings=PEC_SETTINGS)
cal.run(log="flat")

┌──────────────────────────┬───────────────────────────────────────────────────┬────────────────────────────┐
│ benchmark                │ strategy                                          │ performance                │
├──────────────────────────┼───────────────────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: PEC                                    │ ✔                          │
│ Num qubits: 2            │ Representation function: local_depolarizing_noise │ Noisy error: 0.04          │
│ Circuit depth: 2         │ Noise level: 0.001                                │ Mitigated error: 0.0044    │
│ Two qubit gate count: 1  │ Noise bias: 0                                     │ Improvement factor: 9.0915 │
│                          │ Is qubit dependent: False                         │                            │
│                          │ Num samples: 200                                  │                            │
├─────────

In [51]:
cal.best_strategy()

{'technique': 'PEC', 'representation_function': 'represent_operation_with_local_depolarizing_noise', 'noise_level': 0.001, 'noise_bias': 0, 'is_qubit_dependent': False, 'num_samples': 200}